# OpenFHE (Full Source Build) + Streamlit on Google Colab

This notebook shows **full source build** steps for OpenFHE (C++) and the OpenFHE Python wrapper on Google Colab. **Building from source is heavy** (CPU/time/memory). Use a high-RAM Colab runtime if possible. Expect 20–40+ minutes depending on instance.

**What this notebook will do:**
1. Install system build tools (cmake, build-essential, etc.)
2. Clone and build OpenFHE (C++) from GitHub
3. Install the Python wrapper (`openfhe-python`) by building its wheel
4. Install Streamlit and runtime requirements
5. Provide a minimal Streamlit app and instructions to run it (and expose via `ngrok`)

> NOTE: Running this will consume compute and may not complete on small Colab instances. If `pip install openfhe` works, prefer that fast path.


In [ ]:
# 1) Install system prerequisites (apt-get)
# This cell installs build tools, cmake, git, and other libs needed to compile OpenFHE.
# It may ask for sudo; Colab grants sudo in its environment.

import os
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

!apt-get update -y
!apt-get install -y build-essential cmake git wget unzip python3-dev python3-pip libomp-dev pkg-config libssl-dev libbz2-dev liblzma-dev

# Upgrade pip and install wheel
!pip install --upgrade pip setuptools wheel


In [ ]:
# 2) Clone OpenFHE and build from source
# This step compiles the OpenFHE C++ library. It takes time.

# Choose a tag/branch if you want a stable release, e.g., v1.4.0. Using main may track latest.
OPENFHE_REPO='https://github.com/openfheorg/openfhe-development.git'
OPENFHE_DIR='openfhe-development'

if not os.path.exists(OPENFHE_DIR):
    !git clone --depth 1 {OPENFHE_REPO} {OPENFHE_DIR}

# Create build directory and run CMake + make
import multiprocessing
njobs = max(1, min(4, multiprocessing.cpu_count()//2))
print('Using', njobs, 'parallel jobs for make')

!mkdir -p {OPENFHE_DIR}/build
%cd {OPENFHE_DIR}/build
!cmake -DCMAKE_BUILD_TYPE=Release ..
!make -j{njobs}
# Optionally install (requires sudo)
!make install
%cd -


In [ ]:
# 3) Build and install the OpenFHE Python wrapper (openfhe-python)
# This builds a wheel from the wrapper sources and installs it.

WRAPPER_REPO='https://github.com/openfheorg/openfhe-python.git'
WRAPPER_DIR='openfhe-python'

if not os.path.exists(WRAPPER_DIR):
    !git clone --depth 1 {WRAPPER_REPO} {WRAPPER_DIR}

%cd {WRAPPER_DIR}
!pip install -e .
%cd -

# Verify import
print('Testing import...')
try:
    import openfhe
    print('openfhe imported successfully, version:', getattr(openfhe, '__version__', 'unknown'))
except Exception as e:
    print('Import failed:', e)
    raise


In [ ]:
# 4) Install runtime Python dependencies (Streamlit, pandas, plotly, etc.)
!pip install streamlit pandas plotly scikit-learn pyngrok


In [ ]:
# 5) Create a minimal Streamlit app that verifies OpenFHE is present.
# You can replace this with your full 'app_extended.py' by uploading or copy-pasting into Colab.

app_code = r"""
import streamlit as st
st.set_page_config(page_title='OpenFHE Colab Demo')
st.title('OpenFHE Colab — health check')

try:
    import openfhe
    st.success('openfhe imported successfully')
    st.write('OpenFHE module:', openfhe)
except Exception as e:
    st.error('Failed to import openfhe: ' + str(e))

st.write('This is a minimal placeholder app. To run the extended Streamlit app, upload your app_extended.py using the file upload cell or paste the file contents into a new cell and save to app_extended.py')
"""

with open('app_colab_minimal.py', 'w') as f:
    f.write(app_code)

print('Wrote app_colab_minimal.py')


In [ ]:
# 6) (Optional) Upload your full app_extended.py here if you have it locally to Colab
# Use this cell's file chooser to upload. After uploading, the file will be saved as app_extended.py
from google.colab import files
print('Upload app_extended.py if you want to run the full app (press Cancel to skip)')

uploaded = files.upload()
for fn in uploaded:
    print('Uploaded file:', fn)
    # already saved in working directory

print('\nFiles in working dir:')
!ls -la


### Running Streamlit and exposing via ngrok

To run the Streamlit app and expose it publicly from Colab you can use `pyngrok`.

Example commands (run in separate cells):

```python
# 1) Start the Streamlit app (in background)
!nohup streamlit run app_colab_minimal.py --server.port=8501 --server.headless true &

# 2) Install pyngrok and create a tunnel
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print('Public URL:', public_url)
```

**Important:** ngrok free tier connections expire and may require auth token. For persistent use, provide your ngrok authtoken:

```python
!ngrok authtoken YOUR_NGROK_AUTH_TOKEN
```

If you uploaded `app_extended.py`, replace `app_colab_minimal.py` with `app_extended.py` in the run command.


---

## Final notes & troubleshooting

- Building OpenFHE from source on Colab can fail if the instance runs out of memory. If you see `Killed` during `make`, try reducing parallel jobs or use a larger instance.
- If `pip install -e openfhe-python` fails, inspect the error logs — common reasons: missing headers, incompatible compiler, or the OpenFHE library not installed in a standard location. You may need to set `LD_LIBRARY_PATH` or install to `/usr/local` via `sudo make install`.
- If the `openfhe` import fails after installation, run `!ldconfig` (requires sudo) or configure `LD_LIBRARY_PATH` to include the folder where OpenFHE installed its shared libraries (commonly `/usr/local/lib`).

If you'd like, I can now:
- produce the `.ipynb` file for you to download, or
- run these steps here and share logs (note: I cannot run Colab here; you should run the notebook in Google Colab)

